In [ ]:
import moldesign as mdt
import moldesign.units as u

This notebook demonstrates how to prepare an experimental crystal structure for simulation. We'll work with the crystal structure of HIV-1 protease (PDB:3AID) in complex with a small organic molecule.

This structure demonstrates two common obstacles to simulation:
 * a non-biochemical component (the small molecule inhibitor), and
 * histidine residues.


# The crystal structure
First, we'll read in the file and take a look at its contents. As you can see, there are two chains, consisting mostly of protein residues with some water mixed in. Chain `A` also includes an unknown residue, `ARQ401`.

In [ ]:
mol = mdt.read('data/3AID.pdb')
mol

Next, we'll draw the molecule. By default, the protein polymers will be drawn as ribbons, water molecules as lines, and the drug molecule in the "stick" representation.

Next, let's try to assign a forcefield to this structure - the procedure will fail (it show raise a `ParameterizationError` Exception), but we'll get some useful information. When the calcualtion is finished, click on the "ERRORS/WARNINGS" tab to see why it failed.

In [ ]:
newmol = mdt.assign_forcefield(mol)

You should see 3 errors: 
 * The residue name `ARQ` not recognized
 * Atom `HD1` in residue `HIS69`, chain `A` was not recognized
 * Atom `HD1` in residue `HIS69`, chain `B` was not recognized
 
(There's also a warning about bond distances, but these can be generally be fixed with an energy minimization before running dynamics)

We'll deal with the histidine residues first.

## Dealing with histidine
Histidine is notoriously tricky, because it exists in no less than three different protonation states at biological pH (7.4) - the "delta-protonated" form, referred to with residue name `HID`; the "epsilon-protonated" form aka `HIE`; and the doubly-protonated form `HIP`, which has a +1 charge. These are drawn below.

In [ ]:
hid = mdt.from_smiles('O=C(O)[C@@H](N)Cc1c[nH]cn1')
hid.draw3d()

In [ ]:
hie = mdt.from_smiles('O=C(O)[C@@H](N)Cc1cnc[nH]1')
hie.draw3d()

Crystallographers usually can't resolve hydrogen positions, and so can't tell you which form of histidine you have. PDB files thus refer to histidine as `HIS`, which leaves the protonation state ambiguous. In general, picking the right protontation is both extremely important and extremely difficult.

We're in luck for this simulation, however - the histidines are located on the surface of the protein, quite far from the drug binding site, and so their state is not likely to be important for a drug binding calculation. Further, the presence of the `HD1` atoms in the crystal structure indicates that we should probably go ahead and change our `HIS` residues to `HID`.

The fix is easy - just change the name. We'll rerun the paramterization to confirm that the histidine errors disappeared (although the small molecule errors remain).

In [ ]:
for residue in mol.residues:
    if residue.resname == 'HIS':
        residue.resname = 'HID'
        print 'I changed %s in chain %s to HID' % (residue.name, residue.chain.name)

In [ ]:
md_ready_molecule = mdt.forcefield.assign_forcefield(mol)

## Focusing on the protein
First, we'll just run dynamics without the small molecule at all - for instance, maybe we're just interested in the dynamical conformations of the protein in its unbound state. We'll create a new molecule using only the amino acid atoms, parameterize it, and run some preliminary dynamics.

In [ ]:
receptor_structure = mdt.Molecule([atom for atom in mol.atoms if atom.residue.type == 'protein'])
receptor = mdt.assign_forcefield(receptor_structure)

In [ ]:
debug

In [ ]:
model = mdt.models.OpenMMPotential()
print model.print_parameters()

In [ ]:
model.params.implicit_solvent = 'obc'
model.cutoff = 8.0 * u.angstrom
receptor.set_energy_model(model)

In [ ]:
integrator = mdt.integrators.OpenMMLangevin(timestep=2.0*u.fs)
receptor.set_integrator(integrator)

#### 1) Minimize the entire structure

In [ ]:
receptor.draw()

In [ ]:
min1 = receptor.minimize()

In [ ]:
min1.draw()

#### 2) Freeze the protein backbone and heat the system at 300 K
We might consider slowly heating the system in production, but we'll jump to 300 K for now.

In [ ]:
receptor.write('recpt.pkl.bz2')

In [ ]:
receptor = mdt.read('recpt.pkl.bz2')

In [ ]:
for residue in receptor.residues:
    for atom in residue.backbone:
        receptor.constrain_atom(atom)
print 'Constrained %d atoms' % len(receptor.constraints)

In [ ]:
warmup = receptor.run(20.0*u.ps)

In [ ]:
receptor.clear_constraints()
receptor.integrator.params.frame_interval=0.1*u.ps
equil = receptor.run(5.0*u.ps)

In [ ]:
equilmovie = equil.draw()
sidechains = []
for res in receptor.residues: sidechains += [atom for atom in res.sidechain if atom.atnum != 1]
for residue in receptor.residues: sidechains.append(residue['CA'])
equilmovie.cartoon()
equilmovie.color_by(lambda x:float(x.index))
equilmovie.add_style(style='licorice',atoms=sidechains, opacity=0.8)
equilmovie

## Dealing with small molecules
Small molecules present a different set of challenges - the commonly used CHARMM- and AMBER-type force fields only contain parameters for biopolymers.

In this notebook, we'll use the GAFF (generalized Amber force field), which was designed to address this very problem. We'll:
 1. Assign GAFF force field terms for bonded and dispersive (Lennard-Jones) interactions
 2. Calculate partial electrostatic charges that reproduce the molecule's electric field
 
First, let's isolate the drug molecule and add hydrogens to it. Click on the drug residue in the following widget:

In [ ]:
!pwd

In [ ]:
!cp MD\ Setup_new.ipynb /opt/moldesign/notebooks/

In [ ]:
sel = bb.ui.ResidueSelector(mol)
sel

In [ ]:
drugres = sel.selected_residues[0]

In [ ]:
drugmol = bb.add_hydrogen(bb.Molecule(drugres)) # TODO: unique hydrogen names
drugmol.draw()

In [ ]:
drugmol.draw2d(show_hydrogens=True, width=700)

For a production-level calculation, you would need to A) perform a quantum chemical minimization at the RHF/6-31g\* level, B) calculate a set of electrostatic potentials around the molecule, and C) use the RESP procedure to fit atomic charges to the molecule that reproduce the calculated potential.

For this tutorial, however, we'll just use a procedure called AM1-BCC; it's less accurate but MUCH faster.

Also, now's a good time to think about formal charge. Note the quaternary nitrogen (`N2`) has a formal charge of +1; we should check to make sure

In [ ]:
assert drugmol.charge == +1

In [ ]:
result = drugmol.get_gaff_parameters(charge='am1-bcc', )

In [ ]:
print drugmol.write(format='pdb')

In [ ]:
mol = bb.from_smiles('C1=CCNOC1')

In [ ]:
#include commands as file ... stdout, stderr in main display

In [ ]:
reload(bb.interfaces.ambertools)
bb.interfaces.ambertools.am1_bcc_charges(mol)